**UAS BIG DATA RFM RAHIL**

In [43]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt

In [44]:
datatrx = pd.read_csv('../input/rahil-uas-rfm/UAS_RFM.csv', sep=';', parse_dates=['TrxDate'])
datatrx.head()

,CardID,TrxDate,Amount
0,1000152,2021-04-14,822138
1,1000158,2021-02-13,402984
2,1000156,2021-04-05,291013
3,1000115,2021-06-21,968713
4,1000164,2021-03-06,262412


In [45]:
datatrx.head(11)

,CardID,TrxDate,Amount
0,1000152,2021-04-14,822138
1,1000158,2021-02-13,402984
2,1000156,2021-04-05,291013
3,1000115,2021-06-21,968713
4,1000164,2021-03-06,262412
5,1000136,2021-05-30,530683
6,1000147,2021-01-31,423410
7,1000118,2021-09-03,962578
8,1000167,2021-01-18,484129
9,1000160,2021-01-20,524607


In [46]:
datatrx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   CardID   199 non-null    int64         
 1   TrxDate  199 non-null    datetime64[ns]
 2   Amount   199 non-null    int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 4.8 KB


In [47]:
print(datatrx['TrxDate'].min(), datatrx['TrxDate'].max())

2021-01-01 00:00:00 2021-12-06 00:00:00


In [48]:
sd = dt.datetime(2021,12,18)
datatrx['hist']=sd - datatrx['TrxDate']
datatrx['hist'].astype('timedelta64[D]')
datatrx['hist']=datatrx['hist'] / np.timedelta64(1, 'D')
datatrx.head(11)

,CardID,TrxDate,Amount,hist
0,1000152,2021-04-14,822138,248.0
1,1000158,2021-02-13,402984,308.0
2,1000156,2021-04-05,291013,257.0
3,1000115,2021-06-21,968713,180.0
4,1000164,2021-03-06,262412,287.0
5,1000136,2021-05-30,530683,202.0
6,1000147,2021-01-31,423410,321.0
7,1000118,2021-09-03,962578,106.0
8,1000167,2021-01-18,484129,334.0
9,1000160,2021-01-20,524607,332.0


In [49]:
datatrx=datatrx[datatrx['hist'] < 730]
datatrx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199 entries, 0 to 198
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   CardID   199 non-null    int64         
 1   TrxDate  199 non-null    datetime64[ns]
 2   Amount   199 non-null    int64         
 3   hist     199 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 7.8 KB


In [50]:
rfmTable = datatrx.groupby('CardID').agg({'hist': lambda x:x.min(), # Recency
                                        'CardID': lambda x: len(x), # Frequency
                                        'Amount': lambda x: x.sum()}) # Monetary Value

rfmTable.rename(columns={'hist': 'recency', 
                         'CardID': 'frequency', 
                         'Amount': 'monetary'}, inplace=True)
rfmTable.head(11)

,recency,frequency,monetary
CardID,,,
1000101,247.0,1,842222
1000102,137.0,3,1397276
1000103,351.0,1,370038
1000104,211.0,2,946188
1000105,14.0,5,2383497
1000106,299.0,2,904283
1000107,241.0,3,1238289
1000108,263.0,3,1807726
1000109,211.0,2,871162


In [51]:
quartiles = rfmTable.quantile(q=[0.20,0.40,0.60])
print(quartiles, type(quartiles))

     recency  frequency   monetary
0.2     78.0        1.0   788263.0
0.4    178.0        2.0   985684.0
0.6    209.0        3.0  1480542.0 <class 'pandas.core.frame.DataFrame'>


In [52]:
def RClass(x,p,d):
    if x <= d[p][0.20]:
        return 4
    elif x <= d[p][0.40]:
        return 3
    elif x <= d[p][0.60]: 
        return 2
    else:
        return 1
    

In [53]:
def FMClass(x,p,d):
    if x <= d[p][0.20]:
        return 1
    elif x <= d[p][0.40]:
        return 2
    elif x <= d[p][0.60]: 
        return 3
    else:
        return 

In [54]:
rfmSeg = rfmTable
rfmSeg['R_Quartile'] = rfmSeg['recency'].apply(RClass, args=('recency',quartiles,))
rfmSeg['F_Quartile'] = rfmSeg['frequency'].apply(FMClass, args=('frequency',quartiles,))
rfmSeg['M_Quartile'] = rfmSeg['monetary'].apply(FMClass, args=('monetary',quartiles,))

In [55]:
rfmSeg['RFMClass'] = rfmSeg.R_Quartile.map(str) \
                            + rfmSeg.F_Quartile.map(str) \
                            + rfmSeg.M_Quartile.map(str)
rfmSeg.head(11)

,recency,frequency,monetary,R_Quartile,F_Quartile,M_Quartile,RFMClass
CardID,,,,,,,
1000101,247.0,1,842222,1,1.0,2.0,11.02.0
1000102,137.0,3,1397276,3,3.0,3.0,33.03.0
1000103,351.0,1,370038,1,1.0,1.0,11.01.0
1000104,211.0,2,946188,1,2.0,2.0,12.02.0
1000105,14.0,5,2383497,4,NaN,NaN,4nannan
1000106,299.0,2,904283,1,2.0,2.0,12.02.0
1000107,241.0,3,1238289,1,3.0,3.0,13.03.0
1000108,263.0,3,1807726,1,3.0,NaN,13.0nan
1000109,211.0,2,871162,1,2.0,2.0,12.02.0


In [56]:
rfmSeg['RFMClass'] = rfmSeg.R_Quartile.map(str) \
                            + rfmSeg.F_Quartile.map(str) \
                            + rfmSeg.M_Quartile.map(str)
rfmSeg.head(11)

,recency,frequency,monetary,R_Quartile,F_Quartile,M_Quartile,RFMClass
CardID,,,,,,,
1000101,247.0,1,842222,1,1.0,2.0,11.02.0
1000102,137.0,3,1397276,3,3.0,3.0,33.03.0
1000103,351.0,1,370038,1,1.0,1.0,11.01.0
1000104,211.0,2,946188,1,2.0,2.0,12.02.0
1000105,14.0,5,2383497,4,NaN,NaN,4nannan
1000106,299.0,2,904283,1,2.0,2.0,12.02.0
1000107,241.0,3,1238289,1,3.0,3.0,13.03.0
1000108,263.0,3,1807726,1,3.0,NaN,13.0nan
1000109,211.0,2,871162,1,2.0,2.0,12.02.0


In [57]:
print("Pelanggan Tetap: ",len(rfmSeg[rfmSeg['RFMClass']=='444']))
print('Berlangganan: ',len(rfmSeg[rfmSeg['F_Quartile']==4]))
print("Pemborong: ",len(rfmSeg[rfmSeg['M_Quartile']==4]))
print("Beresiko: ",len(rfmSeg[rfmSeg['R_Quartile']==1]))
print('Nyaris Kehilangan Pelanggan: ', len(rfmSeg[rfmSeg['RFMClass']=='111']))
print('Kehilangan Pelanggan: ', len(rfmSeg[rfmSeg['RFMClass']=='333']) + len(rfmSeg[rfmSeg['RFMClass']=='444']))
print('Kehilangan Pelanggan Hemat: ', len(rfmSeg[rfmSeg['RFMClass']=='222']) + len(rfmSeg[rfmSeg['RFMClass']=='111']))

Pelanggan Tetap:  0
Berlangganan:  0
Pemborong:  0
Beresiko:  28
Nyaris Kehilangan Pelanggan:  0
Kehilangan Pelanggan:  0
Kehilangan Pelanggan Hemat:  0
